In [1]:
from llm_utils import *

In [19]:
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "mysecretpassword"
HOST = "localhost"
PORT = 5432
TABLES = []  # list of tables to load or [] to load all tables

from db_connectors import PostgresConnector
from prompt_formatters import RajkumarFormatter, SqlCoderFormatter

try:
    # Get the connector and formatter
    postgres_connector = PostgresConnector(
        user=USER, password=PASSWORD, dbname=DATABASE, host=HOST, port=PORT
    )
    postgres_connector.connect()
    if len(TABLES) <= 0:
        TABLES.extend(postgres_connector.get_tables())
    
    print(f"Loading tables: {TABLES}")
    
    db_schema = [postgres_connector.get_schema(table) for table in TABLES]
    formatter = SqlCoderFormatter(db_schema)
except:
    formatter = SqlCoderFormatter([])
    # Reading from the file
    with open("sample_table_str_raw.txt", 'r') as file:
        formatter.table_str = file.read()

In [20]:
question = "How many products are we selling ?"


In [21]:
%%time
# Enrich the db definition with comments
formatter.table_str = add_comment_to_sql(formatter.table_str, debug=False)
# Build the prompt for Text2SQL
prompt = formatter.format_prompt(question, database='Postgres')
# Get SQLQuery
response = prompt_llm_offline(prompt=prompt, max_length=2500, model_name=HG_SQL_LLM_MODEL_PATH)
query = extract_sql_code(response)

Loading model...
Using device cuda.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tokenizing....
Generating...
Loading model...
Using device cuda.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Tokenizing....
Generating...
CPU times: user 2min 17s, sys: 1min 35s, total: 3min 53s
Wall time: 1min 46s


In [22]:
print(query)


-- Replace the query below with your SQL query:
 SELECT COUNT(DISTINCT products.productid) AS number_of_products FROM products;



In [8]:
print(response)

### Instructions:
Your task is convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the questioq `What is the most frequently used payment method ?`.
This query will run on a database whose schema is represented in this string:

CREATE TABLE suppliers (
    supplierid SERIAL PRIMARY KEY, -- supplierid is the unique identifier for each supplier
    suppliername VARCHAR(100), -- supplier name
    contactname VARCHAR(100), -- contact person's name
    address VARCHAR(255), -- supplier's address
    city VARCHAR(50), -- city where the supplier is located
    country VARCHAR(50), -- country wher